In [78]:
import re
import torch
import json

from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain import HuggingFacePipeline

#from stockfish import Stockfish

In [4]:
bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_use_double_quant=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.bfloat16
)

In [5]:
#model_id = "meta-llama/Llama-2-7b-chat-hf"
#model_id = "mistralai/Mistral-7B-Instruct-v0.1"
model_id = "meta-llama/Llama-2-13b-chat-hf"
#model_id = "HuggingFaceH4/zephyr-7b-beta"

model = AutoModelForCausalLM.from_pretrained(model_id,
                                            quantization_config=bnb_config,
                                            load_in_4bit=True,
                                            use_cache=False, 
                                            device_map="auto")

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards: 100%|██████████| 3/3 [00:22<00:00,  7.46s/it]


In [233]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer, 
    max_length=512,
    temperature=0.00001,
    #top_p=0.15,
    #repetition_penalty=1
)

local_llm = HuggingFacePipeline(pipeline=pipe)

In [234]:
def init_json_chat_history():
    json_chat_history = {"Opponent": [], "Carlus Magnusen": []}

    return json_chat_history

In [235]:
def build_chat_history(message, json_chat_history, bot=3):

    if bot == 0:
        json_chat_history["Opponent"].append(message)
    elif bot == 1:
        json_chat_history["Carlus Magnusen"].append(message)

    formatted_dialogue_list = []

    for i in range(len(json_chat_history["Opponent"])):
        formatted_dialogue_list.append(f'{json_chat_history["Opponent"][i]}')
        if i < len(json_chat_history["Carlus Magnusen"]):
            formatted_dialogue_list.append(f'{json_chat_history["Carlus Magnusen"][i]}')

        
    formatted_chat_history = ""
    for i, formatted_dialogue in enumerate(formatted_dialogue_list):
        if i % 2 == 0:
            formatted_chat_history = formatted_chat_history + formatted_dialogue + " [/INST]"
        else:
            formatted_chat_history = " " + formatted_chat_history + formatted_dialogue + "</s><s>[INST] "

    #formatted_chat_history = '\n'.join(formatted_dialogue_list)

    return json_chat_history, formatted_chat_history

In [237]:
def retrieve_json(text):
    json_string = re.search(r'\{.*\}', text).group()
    loaded_json = json.loads(json_string)
    _, json_value = list(loaded_json.items())[0]
    return json_value

In [281]:
def build_prompt(prompt, formatted_chat_history, tool, best_move=None):

#     template = f"""[INST] <<SYS>>
# You are a helpful, respectful and honest assistant named Carlus Magnusen.
# Act in a professional manner and keep your answers short.
# Bellow you will find the best chess move, you should tell your opponent that this is the move you play.
# Best chess move: e6
# <</SYS>>
# {formatted_chat_history}[/INST] 
# """

    if tool == "classifier":
        system_message = """Your objective is to classify the user input.
There are two possible calssifications: [chess move, no chess move]
If the user is playing a chess move against you, your classification should be "chess move" even if it is an ilegal move.
If the user is not playing a chess move against you, your classification should be "no chess move".
your answer must be as short as possible and have a json in the following format:
{"classification": "<<classification you chose>>"}"""

        full_prompt = f"""<s>[INST] <<SYS>>
{system_message}
<</SYS>>

{prompt} [/INST]"""
        
    if tool == "chess_move_extractor":
        system_message = """Your objective is to extract the chess move that the user played against you.
your answer must be justified and have a json in the following format:
{"move": "<<chess move you extracted in chess algebraic notation>>"}"""

        full_prompt = f"""<s>[INST] <<SYS>>
{system_message}
<</SYS>>

{prompt} [/INST]"""

    if tool == "chess_bot":
        system_message = f"""You are Carlus Magnusen and your purpose is to play the chess move {best_move}
Do not explain the reasoning behind your move.
Act in a professional manner and keep your answers as short as possible."""

        full_prompt = f"""<s>[INST] <<SYS>>
{system_message}
<</SYS>>

{prompt} [/INST]"""

        full_prompt = f"""<s>[INST] <<SYS>>
{system_message}
<</SYS>>

{formatted_chat_history}"""       
        
    if tool == "chat_bot":
        system_message = f"""You are Carlus Magnusen.
Act in a professional manner and keep your answers as short as possible."""

        full_prompt = f"""<s>[INST] <<SYS>>
{system_message}
<</SYS>>

{prompt} [/INST]"""

    return full_prompt

In [ ]:
<s>[INST] <<SYS>>
{your_system_message}
<</SYS>>

{user_message_1} [/INST] {model_reply_1}</s><s>[INST] {user_message_2} [/INST]

In [187]:
print(formatted_chat_history)

 I play e4 [/INST]  Very well, I shall play e5.

My move is:

1. e5

Your move.</s><s>[INST] 


In [282]:
json_chat_history = init_json_chat_history()
chess_moves = []

In [ ]:
prompt = "I play e4"
best_move = "d5"

prompt = "I take with my pawn on d5"
best_move = "Qxd5"

In [284]:
prompt = "I take with my pawn on d5"

json_chat_history, formatted_chat_history = build_chat_history(prompt, json_chat_history, bot=0)

full_prompt = build_prompt(prompt, formatted_chat_history, "classifier")
classifier_output = local_llm(full_prompt)
classification = retrieve_json(classifier_output)
print("\n" + full_prompt + "\n")
print("\nclassifier_output: " + classifier_output + "\n")
print("\nCLASSIFICATION: " + classification + "\n")

if classification == 'chess move':
    full_prompt = build_prompt(prompt, formatted_chat_history, "chess_move_extractor")
    extractor_output = local_llm(full_prompt)
    chess_move = retrieve_json(extractor_output)
    chess_moves.append(chess_move)
    print("\n" + full_prompt + "\n")
    print("\nextractor_output: " + extractor_output + "\n")
    print("\nCHESS MOVE: " + chess_move + "\n")

    # DO STOCKFISH HERE
    best_move = "Qxd5"
    
    full_prompt = build_prompt(prompt, formatted_chat_history, "chess_bot", best_move)
    output = local_llm(full_prompt)
    json_chat_history, formatted_chat_history = build_chat_history(output, json_chat_history, bot=1)
    print("\n" + full_prompt + "\n")
    print("\n" + output + "\n")

    full_prompt = build_prompt(output, formatted_chat_history, "chess_move_extractor")
    extractor_output = local_llm(full_prompt)
    chess_move = retrieve_json(extractor_output)
    chess_moves.append(chess_move)
    print("\n" + full_prompt + "\n")
    print("\nextractor_output: " + extractor_output + "\n")
    print("\nCHESS MOVE: " + chess_move + "\n")


if classification == 'no chess move':
    full_prompt = build_prompt(prompt, formatted_chat_history, "chat_bot")
    output = local_llm(full_prompt)
    print("\n" + full_prompt + "\n")
    print("\n" + output + "\n")

/home/antonio/anaconda3/envs/chess_bot/lib/python3.11/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



<s>[INST] <<SYS>>
Your objective is to classify the user input.
There are two possible calssifications: [chess move, no chess move]
If the user is playing a chess move against you, your classification should be "chess move" even if it is an ilegal move.
If the user is not playing a chess move against you, your classification should be "no chess move".
your answer must be as short as possible and have a json in the following format:
{"classification": "<<classification you chose>>"}
<</SYS>>

I take with my pawn on d5 [/INST]


classifier_output:   {"classification": "chess move"}


CLASSIFICATION: chess move



/home/antonio/anaconda3/envs/chess_bot/lib/python3.11/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



<s>[INST] <<SYS>>
Your objective is to extract the chess move that the user played against you.
your answer must be justified and have a json in the following format:
{"move": "<<chess move you extracted in chess algebraic notation>>"}
<</SYS>>

I take with my pawn on d5 [/INST]


extractor_output:   Based on your move "I take with my pawn on d5", I can extract the chess move as follows:

{"move": "e4"}

Explanation:

When you played "I take with my pawn on d5", you are capturing my pawn on d5 with your pawn on e4. This move is written in chess algebraic notation as "e4", which is the move that I have extracted.

Therefore, the JSON output would be:

{"move": "e4"}


CHESS MOVE: e4



/home/antonio/anaconda3/envs/chess_bot/lib/python3.11/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



<s>[INST] <<SYS>>
You are Carlus Magnusen and your purpose is to play the chess move Qxd5
Do not explain the reasoning behind your move.
Act in a professional manner and keep your answers as short as possible.
<</SYS>>

 I play e4 [/INST]  Very well. I respond with d5.</s><s>[INST] I take with my pawn on d5 [/INST]


  Very well. I capture your pawn with my knight on d5.



/home/antonio/anaconda3/envs/chess_bot/lib/python3.11/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



<s>[INST] <<SYS>>
Your objective is to extract the chess move that the user played against you.
your answer must be justified and have a json in the following format:
{"move": "<<chess move you extracted in chess algebraic notation>>"}
<</SYS>>

  Very well. I capture your pawn with my knight on d5. [/INST]


extractor_output:   Great! Based on your move, I can extract the chess move as follows:

{"move": "Nxd4"}

This move captures your pawn on d5 with my knight.


CHESS MOVE: Nxd4



In [268]:
chess_moves

['e4', 'e5']

In [278]:
string_test = """<s>[INST] <<SYS>>
Your objective is to extract the chess move that the user played against you.
your answer must be jsutified and have a json in the following format:
{"move": "<<chess move you extracted in chess algebraic notation>>"}
<</SYS>>

I take your pawn with my queen on d5 [/INST]"""

In [279]:
print(local_llm(string_test))

/home/antonio/anaconda3/envs/chess_bot/lib/python3.11/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


  Sure! I can extract the chess move you played against me. Based on your statement, you played the move "Queen takes Pawn on d5".

Here's the JSON format response:

{"move": "Qxd5"}


In [ ]:
def build_prompt(prompt, chat_history):

formated_chat_history = """
"""

    template = """[INST] <<SYS>>
You are a Carlus Magnusen, the best chess player in the known universe and your purpose is to play chess with whoever wants to play with you. You are very confident and arrogant. You trash talk your opponents when you play chess. Bellow you will find the best move, you should always tell your opponent that this is the move you play while taunting him.
Best Move: e6
<</SYS>>
{chat_history}
Opponent: {prompt}[/INST]
Carlus Magnusen: 
"""

return template

In [ ]:
sf_config = {
    "Debug Log File": "",
    "Contempt": 0,
    "Min Split Depth": 0,
    "Threads": 8, # More threads will make the engine stronger, but should be kept at less than the number of logical processors on your computer.
    "Ponder": "false",
    "Hash": 2048, # Default size is 16 MB. It's recommended that you increase this value, but keep it as some power of 2. E.g., if you're fine using 2 GB of RAM, set Hash to 2048 (11th power of 2).
    "MultiPV": 1,
    "Skill Level": 20,
    "Move Overhead": 10,
    "Minimum Thinking Time": 20,
    "Slow Mover": 100,
    "UCI_Chess960": "false",
    "UCI_LimitStrength": "false",
    "UCI_Elo": 9999
}

Stockfish(path="/mnt/c/Users/Antonio/Desktop/Projects/chess_bot/stockfish/stockfish-windows-x86-64-avx2.exe",  depth=22, parameters=sf_config)